In [1]:
import os
import openai
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs, NavigableString, Comment
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
driver = webdriver.Chrome('drivers/chromedriver108')

In [3]:
driver.get('https://www.aircanada.com/us/en/aco/home.html')

In [4]:
doc = bs(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'))

In [7]:
# remove unnecessary tags

for tag_type in ['style', 'script', 'noscript', 'link']:
    tags = doc.find_all(tag_type)
    for tag in tags:
        tag.decompose()

In [10]:
print(doc.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   Air Canada -  Official website
  </title>
  <base href="/"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="max-age=0" http-equiv="cache-control"/>
  <meta content="no-store" http-equiv="cache-control"/>
  <meta content="0" http-equiv="expires"/>
  <meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#fffffe" media="(prefers-color-scheme: light)" name="theme-color"/>
  <meta content="#fffffe" media="(prefers-color-scheme: dark)" name="theme-color"/>
  <!-- generics -->
  <!-- Android -->
  <!-- iOS -->
  <!-- Windows 8 IE 10-->
  <meta content="#000000" name="msapplication-TileColor"/>
  <meta content="/achome/assets/img/favicon/mstile-144x144.png" name="msapplication-TileImage"/>
 </head>
 <body>
  <iframe role="presentation" src="javascript:void(0)" styl

In [26]:
for child in doc.html.children:
    print(child.name)

head
body


In [22]:
doc.body.find_all('form')[0]

<form action="/us/en/aco/home/search-results.html?q=" autocomplete class="ngx-ac-search-form ng-star-inserted" method="get" role="search"><abc-input class="ngx-ac-search-source-header abc-form-element"><abc-form-element-container><div class="abc-form-element-container abc-form-element-material-appearance-borderless abc-form-element-material-appearance-condensed"><div class="abc-form-element-wrapper abc-form-element-material-wrapper abc-form-element-input-wrapper ng-star-inserted"><div class="abc-form-element-flex"><!-- --><div class="abc-form-element-infix"><!-- --><div class="abc-form-element-main-container"><!-- --><!-- --><input aria-disabled="false" aria-invalid="false" aria-label="Search site" autocomplete="off" autocorrect="off" class="abc-form-element-input abc-form-element-main text-transform-none ng-untouched ng-pristine ng-valid ng-star-inserted" id="acSiteSearchInput" name="q" placeholder="" spellcheck="false" type="search"/><!-- --></div><abc-form-element-label class="ng-st

In [17]:
doc.find_all('form')[0].attrs

{'role': 'search',
 'method': 'get',
 'autocomplete': 'off',
 'class': ['ngx-ac-search-form', 'ng-star-inserted'],
 'action': '/us/en/aco/home/search-results.html?q='}

In [13]:
driver.find_element(By.XPATH, '/html')

<selenium.webdriver.remote.webelement.WebElement (session="b87b258179c1e388763caeb0613a0892", element="9ae43bc6-7378-4ff9-a398-5a2215d0224d")>

In [24]:
prompt = '''We have the following html element in page:
{0}
The attributes that give us information about the functionality of this element are:
1.
'''.format(
    doc.body.find_all('form')[1].find_all('input')[4]
)

prompt

'We have the following html element in page:\n<input aria-checked="false" aria-disabled="false" aria-invalid="false" aria-label="Book with Aeroplan points" aria-required="false" class="abc-form-element-main abc-form-element-checkbox ng-untouched ng-pristine ng-valid" data-analytics-track="home magnet" data-analytics-val="flights&gt;redeem ae points&gt;select" id="bkmgFlights_searchTypeToggle" role="checkbox" type="checkbox"/>\nThe attributes that give us information about the functionality of this element are:\n1.\n'

In [25]:
local_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
)

In [27]:
print(local_res.choices[0].text)

aria-checked: This attribute indicates whether the checkbox is checked or not.
2. aria-disabled: This attribute indicates whether the checkbox is disabled or not.
3. aria-invalid: This attribute indicates whether the checkbox is valid or not.
4. aria-label: This attribute provides a label for the checkbox.
5. aria-required: This attribute indicates whether the checkbox is required or not.
6. role: This attribute indicates the role of the element, in this case a checkbox.
7. type: This attribute indicates the type of the element, in this case a checkbox.


NameError: name 'doc' is not defined